In [2]:
from datasets import load_dataset
judge = load_dataset("Suchae/korean-judgment-easyread-transform")

/home/infidea/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def combine_and_split_judgments(dataset):
    # 모든 judgment_chunk를 하나의 문자열로 결합
    combined_text = ' '.join(dataset['judgment_chunk'])
    
    # '【판시사항】'을 구분자로 사용하여 텍스트 분할
    chunks = combined_text.split('【판시사항】')
    
    # 빈 문자열 제거 및 각 청크 앞뒤 공백 제거
    chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
    
    return chunks

In [4]:
split_judgments = combine_and_split_judgments(judge['train'])

In [5]:
import re
def clean_single_text(text):
    
    # 불필요한 줄바꿈 정리
    text = ' '.join(text.split())
    
    # 여러 공백을 하나의 공백으로 변경
    text = ' '.join(text.split())
    
    text = text.replace('\r\n', '')
        
    # 정규식으로 <> 안의 문자열 제거
    text = re.sub(r'<[^>]*>', '', text)
    
    return text

In [6]:
split_judgments_cleaned = [clean_single_text(text) for text in split_judgments if len(clean_single_text(text)) >200 and len(clean_single_text(text)) < 8000]

In [ ]:
business_keywords = [
    '금융', '재무', '회계', '투자', '주식', '사채', '채권',
    '주주', '배당', '자본', '증권', '합병', '인수', '상장', '거래소', '유가증권',
     '부채', '손익', '매출',
    '수익', '파산',  '예금', '융자', '신용'
]
finance_judgement = [text for text in split_judgments_cleaned if sum(keyword in text for keyword in business_keywords) >= 3]

In [8]:
len(finance_judgement)

5611

In [9]:
finance_judgement[:5]

['채무자가 제시한 화의조건의 이행이 객관적으로 불가능하거나 그 이행가능성에 상당한 의심이 있는 경우, 화의개시신청을 기각하여야 하는지 여부(적극) 및 화의조건의 이행가능성 여부를 판단함에 있어 고려하여야 할 사정【결정요지】 화의는 기본적으로 채권의 내용을 변경하는 것으로서 변경된 내용이 이행될 수 있는가의 여부는 화의절차가 진행되기 위한 본질적 요건이므로 채무자가 제시한 화의조건의 이행이 객관적으로 불가능하다고 인정되거나 그 이행가능성에 상당한 의심이 있는 때에는 화의법 제18조 제5호에 의하여 그 화의개시신청을 기각함이 상당한바, 그 때 화의조건의 이행가능성 여부를 판단함에 있어서는 파산원인이 발생하기에 이른 경위, 화의결정 당시의 채무자의 자산과 부채의 불균형의 정도, 사업의 종류에 의한 수익성의 정도 및 확실성, 경제적 신용의 획득 가망성, 인적·물적 담보의 유무 및 그의 담보가치, 화의절차에 의하지 아니하고 행사되는 채권의 액, 별제권자의 동향 등 일체의 사정을 종합적으로 고려하여 판단하여야 한다.【특별항고인】 주식회사 미광건설 (소송대리인 변호사 김용대) 【원심결정】 대구지법 2000. 3. 4.자 99거7 결정 【주문】 【원심결정】 대구지법 2000. 3. 4.자 99거7 결정 【주문】 특별항고를 기각한다. 【이유】 특별항고 이유를 판단한다. 화의는 기본적으로 채권의 내용을 변경하는 것으로서 변경된 내용이 이행될 수 있는가의 여부는 화의절차가 진행되기 위한 본질적 요건이므로 채무자가 제시한 화의조건의 이행이 객관적으로 불가능하다고 인정되거나 그 이행가능성에 상당한 의심이 있는 때에는 화의법 제18조 제5호에 의하여 그 화의개시신청을 기각함이 상당한바, 그 때 화의조건의 이행가능성 여부를 판단함에 있어서는 파산원인이 발생하기에 이른 경위, 화의결정 당시의 채무자의 자산과 부채의 불균형의 정도, 사업의 종류에 의한 수익성의 정도 및 확실성, 경제적 신용의 획득 가망성, 인적·물적 담보의 유무 및 그의 담보가치, 화의절차에 의하지 아니하고 행사되는 채권

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
import random
import os
os.environ["OPENAI_API_KEY"] = "sk-XXX"

In [ ]:
GPT_API_KEY = "sk-XXX"
import openai
client = openai.OpenAI(api_key=GPT_API_KEY)

In [12]:
import pandas as pd
from datasets import load_dataset, Dataset
import random
from litellm import completion, batch_completion
import os
import litellm

In [13]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")


In [16]:
qrys = []
for t in finance_judgement:
    messages = [
    {"content": "Your job is creating quantitative finance questions in fluent Korean. You will be given a part of a text. Make a question based on it. The question should be general and not strictly bound to the given context. Return the question only.", "role": "system"},
    {"content": t, "role": "user"}]
    qrys.append(messages)
# 1. raw text 데이터를 활용한 질문 생성
responses = batch_completion(
    model="gpt-4o-mini-2024-07-18",
    messages = qrys
)

question_resps = [i.choices[0].message.content for i in responses]
total_prompt_tokens_for_q = sum([r.usage.prompt_tokens for r in responses])
total_completion_tokens_for_q = sum([r.usage.completion_tokens for r in responses])

In [18]:
question_resps[:100]

['화의절차에서 채무자가 제시한 화의조건의 이행가능성을 판단할 때 고려해야 할 주요 요소는 무엇인가요?',
 '금융기관이 채권양도 계약을 체결한 후 채무자의 동의 없이 해당 채권을 양도할 수 있는 법적 요건은 무엇인가요?',
 '이사의 직무상 충실 의무와 선량한 관리자의 주의 의무를 위반한 경우, 이사는 어떤 법적 책임을 질 수 있습니까?',
 '채무자가 채권자 불확지를 원인으로 하는 공탁을 하고자 할 때, 어떤 법적 근거를 통해 이를 수행할 수 있는지 설명해 주시겠습니까?',
 '상법 제24조에 따른 명의대여자의 책임이 거래 상대방을 보호하기 위한 것이란 정황을 고려할 때, 명의대여자인 갑, 을, 병이 실제로 나이트클럽을 경영하지 않았음에도 불구하고 그들의 명의를 사용하여 영업상의 외관을 나타낸 사례에 대해, 이와 관련된 법적 책임은 어떻게 판단될 수 있는가?',
 '부동산의 소유권과 임차권이 동일인에게 귀속될 때 임차권이 소멸하지 않는 경우는 어떤 조건이 충족되어야 하는가?',
 '사해방지참가와 관련하여 독립당사자참가의 요건은 무엇인지 설명해 주실 수 있나요?',
 '공용부분과 전유부분의 구분에 대한 법적 판단이 소유권 취득에 미치는 영향은 무엇인가요?',
 "신탁법 제21조에 따른 '신탁사무의 처리상 발생한 권리'의 정의와 적용 여부에 대한 주요 쟁점은 무엇인가요?",
 '인·허가 과정에서 제출하는 각서의 해석에 있어 고려해야 할 요소들은 무엇인가요?',
 '주식의 무상감자와 유상증자가 주주에게 미치는 세무적 영향은 무엇인가요?',
 '수익사업의 정의와 그 판단 기준은 무엇이며, 이에 따라 특정 사업이 수익사업으로 분류되기 위한 요건은 무엇인가요?',
 '부동산 임대사업자가 차입금으로 취득한 부동산의 임대소득을 계산할 때, 어떤 요소들이 필요경비로 인정되는지에 대해 설명해 보세요.',
 '조세회피 목적이 있는 명의신탁을 판단하는 기준은 무엇인가요?',
 "법인세법 관련 거래에서 '특수관계자'의 정의는 어떻게 이루어지며, 이 정의가 세무 처리에 어떤 영향을

In [21]:
# 답변 생성용 prompt 포맷팅
qrys = []
for t in question_resps:
    messages = [
    {"content":"You are a skilled financial expert in Korea. Make a response for the question. DO NOT introduce yourself.","role":"system"},
    { "content": t,"role": "user"}]
    qrys.append(messages)

# 2. 생성된 질문에 대한 답변 생성
responses = batch_completion(
    model="gpt-4o",
    messages = qrys
)
response_resps = [i.choices[0].message.content for i in responses]
total_prompt_tokens_for_a = sum([r.usage.prompt_tokens for r in responses])
total_completion_tokens_for_a = sum([r.usage.completion_tokens for r in responses])

In [22]:
response_resps[:10]

['화의절차에서 채무자가 제시한 화의조건의 이행가능성을 판단할 때 고려해야 할 주요 요소는 다음과 같습니다:\n\n1. **채무자의 재무 상태**: 채무자가 제시하는 재무제표, 손익계산서, 현금 흐름표 등을 검토하여 현재의 재무 상태를 분석합니다. 자산과 부채의 비율, 유동성 비율, 수익성 등을 확인합니다.\n\n2. **현금 흐름**: 채무자의 미래 현금 흐름을 예측하여 화의 조건 이행에 필요한 자금 조달 가능성을 판단합니다. 충분한 영업 현금 흐름을 확보할 수 있는지가 중요합니다.\n\n3. **사업 계획의 실행 가능성**: 향후 사업 계획과 전략이 현실적이며 실행 가능한지 평가합니다. 시장 상황, 경쟁자 분석, 성장 전망 등을 고려해야 합니다.\n\n4. **부채 재조정 가능성**: 채권자와의 협의를 통해 부채 재조정이나 조정 가능성을 평가합니다. 채권자들이 제안을 수용할지 여부도 중요한 요소입니다.\n\n5. **법적 및 규제적 요인**: 해당 화의절차가 법적 혹은 규제적 장애물에 부딪힐 가능성이 있는지 검토합니다.\n\n6. **이전 이행 실적**: 과거에 제시한 화의 조건이 제대로 이행되었는지, 신뢰할 수 있는지를 확인합니다.\n\n7. **관리팀의 역량**: 채무자의 경영진이 화의 계획을 성공적으로 실행할 수 있는 역량과 경험을 갖추고 있는지 평가합니다.\n\n이러한 요소들을 종합적으로 분석하여 채무자의 화의조건 이행 가능성을 평가하는 것이 중요합니다.',
 '채권양도에 대한 법적 요건은 민법과 관련 규정에 따라 다룹니다. 일반적으로 채권은 양도인이 채권을 타인에게 이전할 수 있는 권리를 가지며, 채무자의 동의 없이도 채권을 양도할 수 있습니다. 다만, 몇 가지 중요한 요건과 제한사항이 있습니다.\n\n1. **채권 양도 금지 특약**: 계약서에 채권 양도를 금지하는 특약이 있는 경우, 해당 채권은 동의 없이 양도할 수 없습니다.\n\n2. **공시**: 양도가 제3자에 대한 대항력을 가지기 위해서는 채무자에게 채권 양도의 사실을 통지하거나 채무자

In [23]:
print('total prompt tokens:', total_prompt_tokens_for_q + total_prompt_tokens_for_a)
print('prompt token costs:', round((total_prompt_tokens_for_q + total_prompt_tokens_for_a) / 1000000 * 2.5, 6))
print('total completion tokens:', total_completion_tokens_for_q + total_completion_tokens_for_a)
print('completion token costs:', round((total_completion_tokens_for_q + total_completion_tokens_for_a) / 1000000 * 10, 6))

total prompt tokens: 16616136
prompt token costs: 41.54034
total completion tokens: 2135928
completion token costs: 21.35928


In [ ]:
df = pd.DataFrame({'question': question_resps, 'response': response_resps})

# CSV 파일 저장
df.to_csv("result.csv")


# HuggingFace Hub 업로드 - token에 개인 HuggingFace 토큰을 입력해주시면 됩니다.
result_df = Dataset.from_pandas(df)
result_df.push_to_hub("overfit-brothers/financial_judgement_instructions", token="hf_XXX",private=True)

df.head()

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.88s/it]


,question,response
0,화의절차에서 채무자가 제시한 화의조건의 이행가능성을 판단할 때 고려해야 할 주요 요...,화의절차에서 채무자가 제시한 화의조건의 이행가능성을 판단할 때 고려해야 할 주요 요...
1,금융기관이 채권양도 계약을 체결한 후 채무자의 동의 없이 해당 채권을 양도할 수 있...,채권양도에 대한 법적 요건은 민법과 관련 규정에 따라 다룹니다. 일반적으로 채권은 ...
2,"이사의 직무상 충실 의무와 선량한 관리자의 주의 의무를 위반한 경우, 이사는 어떤 ...","이사의 직무상 충실 의무와 선량한 관리자의 주의 의무를 위반한 경우, 이사는 여러 ..."
3,"채무자가 채권자 불확지를 원인으로 하는 공탁을 하고자 할 때, 어떤 법적 근거를 통...","채무자가 채권자 불확지를 원인으로 공탁을 하고자 할 때, 민법 및 공탁법에 따른 절..."
4,상법 제24조에 따른 명의대여자의 책임이 거래 상대방을 보호하기 위한 것이란 정황을...,상법 제24조에 따른 명의대여자의 책임은 명의를 빌려준 사람이 실제 영업을 경영하지...
